In [ ]:
#heart_attack prediction 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)


Pandas   1.3.1


In [2]:
df =pd.read_csv("heart.csv")
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df.output.value_counts()

1    165
0    138
Name: output, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
# scaling for the features having different distributions and also measured in different units
#based on standard normal distribution
# also can be done in the pipeline
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
columns_toscale =['age','trtbps','thalachh','oldpeak','chol']
df[columns_toscale]=SS.fit_transform(df[columns_toscale])
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0,1.087338,0,0,1,1
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,2,1
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0,0.310912,2,0,2,1
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0,-0.206705,2,0,2,1
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1,-0.379244,2,0,2,1


In [6]:
X,y = df.drop(['output'],axis=1),df.output

In [7]:
cat_vars=X.select_dtypes(include=[object]).columns.values.tolist()
num_vars =X.select_dtypes(exclude=[object]).columns.values.tolist()
num_vars

['age',
 'sex',
 'cp',
 'trtbps',
 'chol',
 'fbs',
 'restecg',
 'thalachh',
 'exng',
 'oldpeak',
 'slp',
 'caa',
 'thall']

In [8]:
num_preprocessing =pipeline.Pipeline([("imputer",impute.SimpleImputer(strategy="mean"))]) #,("scaler",preprocessing.StandardScaler())

cat_preporcessing =pipeline.Pipeline([("imputer",impute.SimpleImputer(strategy='constant', fill_value='missing')),
                                      ("ordinal",preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value = -1))])
tree_prepro = compose.ColumnTransformer(transformers=[('num_data',num_preprocessing,num_vars),('cat_data',cat_preporcessing,cat_vars),],remainder='drop')
tree_prepro

ColumnTransformer(transformers=[('num_data',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs',
                                  'restecg', 'thalachh', 'exng', 'oldpeak',
                                  'slp', 'caa', 'thall']),
                                ('cat_data',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 [])])

In [9]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier
from sklearn.svm           import SVC
from sklearn.linear_model  import LogisticRegression
from sklearn.neighbors     import KNeighborsClassifier

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor

In [10]:
#checking the classifiers
classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":  ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier(),
  "SVM": SVC(),
  "lr":LogisticRegression(),
  "KNN":KNeighborsClassifier(n_neighbors=3)
}

from sklearn import set_config
set_config(display='diagram')

for tree, method in classifiers.items():
    classifiers[tree]=pipeline.Pipeline([("preprocessing", tree_prepro),(tree,method)])

# tree_classifiers["LightGBM"]
classifiers['SVM']

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['age', 'sex', 'cp', 'trtbps',
                                                   'chol', 'fbs', 'restecg',
                                                   'thalachh', 'exng',
                                                   'oldpeak', 'slp', 'caa',
                                                   'thall']),
                                                 ('cat_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  [])])),
                ('SVM', SVC())])

In [11]:
# checking the regressors
Regressors = {
  "Decision Tree": DecisionTreeRegressor(),
  "Extra Trees":   ExtraTreesRegressor(n_estimators=100),
  "Random Forest": RandomForestRegressor(n_estimators=100),
  "AdaBoost":      AdaBoostRegressor(n_estimators=100),
  "Skl GBM":       GradientBoostingRegressor(n_estimators=100),
  "XGBoost":       XGBRegressor(n_estimators=100),
  "LightGBM":      LGBMRegressor(n_estimators=100),
  "CatBoost":      CatBoostRegressor(n_estimators=100),
}

from sklearn import set_config
set_config(display='diagram')

for tree, method in Regressors.items():
    Regressors[tree]=pipeline.Pipeline([("preprocessing", tree_prepro),(tree,method)])
Regressors['Skl GBM']

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['age', 'sex', 'cp', 'trtbps',
                                                   'chol', 'fbs', 'restecg',
                                                   'thalachh', 'exng',
                                                   'oldpeak', 'slp', 'caa',
                                                   'thall']),
                                                 ('cat_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  [])])),
                ('Skl GBM', GradientBoostingRegressor())])

In [12]:
import time
X_train,X_test,y_train,y_test =model_selection.train_test_split(X,y,test_size=0.2,stratify = y,random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(X_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(X_test)

    total_time = time.time() - start_time
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[14:17:54] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.005621
0:	learn: 0.6882194	total: 170ms	remaining: 2m 49s
1:	learn: 0.6834045	total: 174ms	remaining: 1m 26s
2:	learn: 0.6791326	total: 179ms	remaining: 59.6s
3:	learn: 0.6745045	total: 183ms	remaining: 45.7s
4:	learn: 0.6709091	total: 187ms	remaining: 37.3s
5:	learn: 0.6667564	total: 192ms	remaining: 31.8s
6:	learn: 0.6641029	total: 195ms	remaining: 27.6s
7:	learn: 0.6602772	total: 199ms	remaining: 24.6s
8:	learn: 0.6567561	total: 203ms	remaining: 22.4s
9:	learn: 0.6525413	total: 207ms	remaining: 20.5s
10:	learn: 0.6483383	total: 212ms	remaining: 19.1s
11:	learn: 0.6442340	total: 216ms	remaining: 17.8s
12:	learn: 0.6407615	total: 220ms	remaining: 16.7s
13:	learn: 0.6367297	total: 224ms	remaining: 15.8s
14:	learn: 0

167:	learn: 0.3421263	total: 689ms	remaining: 3.41s
168:	learn: 0.3410270	total: 692ms	remaining: 3.4s
169:	learn: 0.3404698	total: 695ms	remaining: 3.39s
170:	learn: 0.3391242	total: 698ms	remaining: 3.38s
171:	learn: 0.3376754	total: 701ms	remaining: 3.38s
172:	learn: 0.3368140	total: 704ms	remaining: 3.37s
173:	learn: 0.3357103	total: 707ms	remaining: 3.36s
174:	learn: 0.3349462	total: 710ms	remaining: 3.35s
175:	learn: 0.3339887	total: 713ms	remaining: 3.34s
176:	learn: 0.3330547	total: 716ms	remaining: 3.33s
177:	learn: 0.3321940	total: 720ms	remaining: 3.32s
178:	learn: 0.3312999	total: 723ms	remaining: 3.32s
179:	learn: 0.3304771	total: 727ms	remaining: 3.31s
180:	learn: 0.3299055	total: 730ms	remaining: 3.3s
181:	learn: 0.3291382	total: 732ms	remaining: 3.29s
182:	learn: 0.3286051	total: 735ms	remaining: 3.28s
183:	learn: 0.3277176	total: 738ms	remaining: 3.27s
184:	learn: 0.3267549	total: 741ms	remaining: 3.27s
185:	learn: 0.3259469	total: 744ms	remaining: 3.26s
186:	learn: 0.

340:	learn: 0.2358352	total: 1.21s	remaining: 2.34s
341:	learn: 0.2351012	total: 1.22s	remaining: 2.34s
342:	learn: 0.2346161	total: 1.22s	remaining: 2.34s
343:	learn: 0.2342253	total: 1.22s	remaining: 2.33s
344:	learn: 0.2338182	total: 1.22s	remaining: 2.33s
345:	learn: 0.2333949	total: 1.23s	remaining: 2.32s
346:	learn: 0.2329908	total: 1.23s	remaining: 2.31s
347:	learn: 0.2326995	total: 1.23s	remaining: 2.31s
348:	learn: 0.2322217	total: 1.24s	remaining: 2.31s
349:	learn: 0.2320789	total: 1.24s	remaining: 2.3s
350:	learn: 0.2317287	total: 1.24s	remaining: 2.29s
351:	learn: 0.2313361	total: 1.24s	remaining: 2.29s
352:	learn: 0.2308972	total: 1.25s	remaining: 2.29s
353:	learn: 0.2302916	total: 1.25s	remaining: 2.28s
354:	learn: 0.2299771	total: 1.25s	remaining: 2.28s
355:	learn: 0.2297056	total: 1.25s	remaining: 2.27s
356:	learn: 0.2295000	total: 1.26s	remaining: 2.27s
357:	learn: 0.2289190	total: 1.26s	remaining: 2.26s
358:	learn: 0.2285798	total: 1.26s	remaining: 2.26s
359:	learn: 0

516:	learn: 0.1792579	total: 1.72s	remaining: 1.6s
517:	learn: 0.1790208	total: 1.72s	remaining: 1.6s
518:	learn: 0.1786536	total: 1.72s	remaining: 1.59s
519:	learn: 0.1784908	total: 1.73s	remaining: 1.59s
520:	learn: 0.1781821	total: 1.73s	remaining: 1.59s
521:	learn: 0.1777980	total: 1.73s	remaining: 1.58s
522:	learn: 0.1776553	total: 1.73s	remaining: 1.58s
523:	learn: 0.1773614	total: 1.74s	remaining: 1.58s
524:	learn: 0.1771516	total: 1.74s	remaining: 1.57s
525:	learn: 0.1768860	total: 1.74s	remaining: 1.57s
526:	learn: 0.1767192	total: 1.75s	remaining: 1.57s
527:	learn: 0.1764382	total: 1.75s	remaining: 1.56s
528:	learn: 0.1762335	total: 1.75s	remaining: 1.56s
529:	learn: 0.1759624	total: 1.76s	remaining: 1.56s
530:	learn: 0.1758472	total: 1.76s	remaining: 1.56s
531:	learn: 0.1755524	total: 1.77s	remaining: 1.55s
532:	learn: 0.1753768	total: 1.77s	remaining: 1.55s
533:	learn: 0.1751795	total: 1.77s	remaining: 1.55s
534:	learn: 0.1750686	total: 1.78s	remaining: 1.54s
535:	learn: 0.

679:	learn: 0.1452643	total: 2.23s	remaining: 1.05s
680:	learn: 0.1449865	total: 2.23s	remaining: 1.05s
681:	learn: 0.1447861	total: 2.24s	remaining: 1.04s
682:	learn: 0.1444618	total: 2.24s	remaining: 1.04s
683:	learn: 0.1442970	total: 2.24s	remaining: 1.03s
684:	learn: 0.1440623	total: 2.25s	remaining: 1.03s
685:	learn: 0.1438786	total: 2.25s	remaining: 1.03s
686:	learn: 0.1437494	total: 2.25s	remaining: 1.02s
687:	learn: 0.1435022	total: 2.25s	remaining: 1.02s
688:	learn: 0.1432638	total: 2.26s	remaining: 1.02s
689:	learn: 0.1431071	total: 2.26s	remaining: 1.02s
690:	learn: 0.1430143	total: 2.27s	remaining: 1.01s
691:	learn: 0.1428164	total: 2.27s	remaining: 1.01s
692:	learn: 0.1425402	total: 2.27s	remaining: 1.01s
693:	learn: 0.1423891	total: 2.27s	remaining: 1s
694:	learn: 0.1421866	total: 2.28s	remaining: 1000ms
695:	learn: 0.1421353	total: 2.28s	remaining: 996ms
696:	learn: 0.1418147	total: 2.28s	remaining: 992ms
697:	learn: 0.1417288	total: 2.29s	remaining: 989ms
698:	learn: 0.

853:	learn: 0.1168502	total: 2.75s	remaining: 470ms
854:	learn: 0.1166461	total: 2.75s	remaining: 467ms
855:	learn: 0.1165387	total: 2.76s	remaining: 464ms
856:	learn: 0.1163129	total: 2.76s	remaining: 461ms
857:	learn: 0.1162525	total: 2.76s	remaining: 457ms
858:	learn: 0.1161956	total: 2.77s	remaining: 454ms
859:	learn: 0.1159210	total: 2.77s	remaining: 451ms
860:	learn: 0.1157525	total: 2.77s	remaining: 448ms
861:	learn: 0.1156456	total: 2.78s	remaining: 445ms
862:	learn: 0.1155779	total: 2.78s	remaining: 441ms
863:	learn: 0.1153784	total: 2.78s	remaining: 438ms
864:	learn: 0.1152329	total: 2.79s	remaining: 435ms
865:	learn: 0.1150070	total: 2.79s	remaining: 432ms
866:	learn: 0.1148282	total: 2.79s	remaining: 429ms
867:	learn: 0.1146578	total: 2.8s	remaining: 426ms
868:	learn: 0.1144807	total: 2.8s	remaining: 422ms
869:	learn: 0.1143420	total: 2.81s	remaining: 419ms
870:	learn: 0.1141985	total: 2.81s	remaining: 416ms
871:	learn: 0.1140231	total: 2.81s	remaining: 413ms
872:	learn: 0.

,Model,Accuracy,Bal Acc.,Time
1,XGBoost,86.885246,86.525974,0.146937
2,CatBoost,86.885246,86.796537,3.642676
3,lr,85.245902,84.740260,0.059913
4,Skl GBM,83.606557,83.495671,0.199102
5,SVM,83.606557,82.954545,0.016993
6,Extra Trees,81.967213,81.439394,0.262125
7,Random Forest,81.967213,81.439394,0.352252
8,AdaBoost,81.967213,81.980519,0.195679
9,LightGBM,81.967213,81.439394,0.060469
10,Decision Tree,80.327869,80.194805,0.015232


In [13]:
# best_model = classifiers[results_ord['Model'].iloc[0]]
# best_model.fit(X_train,y_train)
# test_pred = best_model.predict(X_test)
# print(f"best_model is {results_ord['Model'].iloc[0]}:",best_model.score(X_test,y_test)*100)

[14:18:10] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best_model is XGBoost: 86.88524590163934


In [53]:
df_new=pd.read_csv("heart.csv")
X_new,y_new = df_new.drop(['output'],axis=1),df.output

In [54]:
#univariate selection # selecting the best features
#feature engineering

#apply selectKBest.class to extact top five best features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
best_ft = SelectKBest(score_func = chi2, k=8)
model =best_ft.fit(X_new,y_new)

In [19]:
df_scores=pd.DataFrame(model.scores_)
df_columns =pd.DataFrame(X_new.columns)
feature_scores = pd.concat([df_columns,df_scores],axis=1)
feature_scores.columns=['features','scores']
feature_scores #highest the score 

,features,scores
0,age,23.286624
1,sex,7.576835
2,cp,62.598098
3,trtbps,14.823925
4,chol,23.936394
5,fbs,0.202934
6,restecg,2.978271
7,thalachh,188.320472
8,exng,38.914377
9,oldpeak,72.644253


In [55]:
best_features = feature_scores.nlargest(4,'scores')['features'].tolist()
best_features

['thalachh', 'oldpeak', 'caa', 'cp']

In [56]:
X_new = X_new[best_features]
X_new.tail()

,thalachh,oldpeak,caa,cp
298,123,0.2,0,0
299,132,1.2,0,3
300,141,3.4,2,0
301,115,1.2,1,0
302,174,0.0,1,1


In [57]:
cat_vars_n=X_new.select_dtypes(include=[object]).columns.values.tolist()
num_vars_n =X_new.select_dtypes(exclude=[object]).columns.values.tolist()
num_preprocessing_n =pipeline.Pipeline([("imputer",impute.SimpleImputer(strategy="mean"))]) #,("scaler",preprocessing.StandardScaler())

cat_preporcessing_n=pipeline.Pipeline([("imputer",impute.SimpleImputer(strategy='constant', fill_value='missing')),
                                      ("ordinal",preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value = -1))])
tree_prepro_n = compose.ColumnTransformer(transformers=[('num_data',num_preprocessing,num_vars_n),('cat_data',cat_preporcessing,cat_vars_n),],remainder='drop')


In [58]:
classifiers_n = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":  ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier(),
  "SVM": SVC(),
  "lr":LogisticRegression(),
  "KNN":KNeighborsClassifier(n_neighbors=10)
}

from sklearn import set_config
set_config(display='diagram')

for tree, method in classifiers_n.items():
    classifiers_n[tree]=pipeline.Pipeline([("preprocessing", tree_prepro_n),(tree,method)])

# tree_classifiers["LightGBM"]
classifiers_n['SVM']

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['thalachh', 'oldpeak', 'caa',
                                                   'cp']),
                                                 ('cat_data',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  [])])),
                ('SVM', SVC())])

In [59]:
x_train,x_test,Y_train,Y_test =model_selection.train_test_split(X_new,y_new,test_size=0.2,stratify = y,random_state=0)

results_new = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in classifiers_n.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,Y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_test)# CODE HERE

    total_time = time.time() - start_time
    
    results_new = results_new.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(Y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(Y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


### BEGIN SOLUTION

results_ord_new = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord_new.index += 1 
results_ord_new.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[14:27:27] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.005621
0:	learn: 0.6888520	total: 1.21ms	remaining: 1.21s
1:	learn: 0.6847507	total: 2.39ms	remaining: 1.19s
2:	learn: 0.6801527	total: 3.72ms	remaining: 1.24s
3:	learn: 0.6762914	total: 4.86ms	remaining: 1.21s
4:	learn: 0.6725207	total: 6.25ms	remaining: 1.24s
5:	learn: 0.6713654	total: 7.07ms	remaining: 1.17s
6:	learn: 0.6680036	total: 8.73ms	remaining: 1.24s
7:	learn: 0.6660239	total: 10.2ms	remaining: 1.26s
8:	learn: 0.6624949	total: 11.3ms	remaining: 1.24s
9:	learn: 0.6599190	total: 12.2ms	remaining: 1.21s
10:	learn: 0.6560067	total: 13.5ms	remaining: 1.21s
11:	learn: 0.6519557	total: 15.4ms	remaining: 1.26s
12:	learn: 0.6489241	total: 17ms	remaining: 1.29s
13:	learn: 0.6462824	total: 18.6ms	remaining: 1.31s
14

184:	learn: 0.3948546	total: 331ms	remaining: 1.46s
185:	learn: 0.3940504	total: 333ms	remaining: 1.46s
186:	learn: 0.3931062	total: 335ms	remaining: 1.46s
187:	learn: 0.3920084	total: 337ms	remaining: 1.46s
188:	learn: 0.3914775	total: 339ms	remaining: 1.45s
189:	learn: 0.3908663	total: 341ms	remaining: 1.45s
190:	learn: 0.3903451	total: 343ms	remaining: 1.45s
191:	learn: 0.3895364	total: 346ms	remaining: 1.45s
192:	learn: 0.3891658	total: 347ms	remaining: 1.45s
193:	learn: 0.3888090	total: 349ms	remaining: 1.45s
194:	learn: 0.3880502	total: 351ms	remaining: 1.45s
195:	learn: 0.3873174	total: 354ms	remaining: 1.45s
196:	learn: 0.3870407	total: 355ms	remaining: 1.45s
197:	learn: 0.3865218	total: 357ms	remaining: 1.45s
198:	learn: 0.3856662	total: 359ms	remaining: 1.45s
199:	learn: 0.3853282	total: 361ms	remaining: 1.44s
200:	learn: 0.3845949	total: 363ms	remaining: 1.44s
201:	learn: 0.3839126	total: 365ms	remaining: 1.44s
202:	learn: 0.3835874	total: 367ms	remaining: 1.44s
203:	learn: 

347:	learn: 0.3220341	total: 653ms	remaining: 1.22s
348:	learn: 0.3216363	total: 655ms	remaining: 1.22s
349:	learn: 0.3212581	total: 657ms	remaining: 1.22s
350:	learn: 0.3208850	total: 658ms	remaining: 1.22s
351:	learn: 0.3206694	total: 660ms	remaining: 1.21s
352:	learn: 0.3204549	total: 661ms	remaining: 1.21s
353:	learn: 0.3199657	total: 663ms	remaining: 1.21s
354:	learn: 0.3195438	total: 664ms	remaining: 1.21s
355:	learn: 0.3193914	total: 666ms	remaining: 1.2s
356:	learn: 0.3192338	total: 668ms	remaining: 1.2s
357:	learn: 0.3186930	total: 669ms	remaining: 1.2s
358:	learn: 0.3184855	total: 670ms	remaining: 1.2s
359:	learn: 0.3183108	total: 672ms	remaining: 1.19s
360:	learn: 0.3181277	total: 673ms	remaining: 1.19s
361:	learn: 0.3179491	total: 675ms	remaining: 1.19s
362:	learn: 0.3178687	total: 676ms	remaining: 1.19s
363:	learn: 0.3176313	total: 677ms	remaining: 1.18s
364:	learn: 0.3173688	total: 680ms	remaining: 1.18s
365:	learn: 0.3172522	total: 682ms	remaining: 1.18s
366:	learn: 0.31

524:	learn: 0.2825778	total: 996ms	remaining: 901ms
525:	learn: 0.2824416	total: 999ms	remaining: 900ms
526:	learn: 0.2824323	total: 1s	remaining: 898ms
527:	learn: 0.2822653	total: 1s	remaining: 896ms
528:	learn: 0.2818878	total: 1s	remaining: 894ms
529:	learn: 0.2813830	total: 1s	remaining: 892ms
530:	learn: 0.2811147	total: 1.01s	remaining: 889ms
531:	learn: 0.2810255	total: 1.01s	remaining: 887ms
532:	learn: 0.2807319	total: 1.01s	remaining: 885ms
533:	learn: 0.2806284	total: 1.01s	remaining: 886ms
534:	learn: 0.2805945	total: 1.02s	remaining: 883ms
535:	learn: 0.2803024	total: 1.02s	remaining: 881ms
536:	learn: 0.2801649	total: 1.02s	remaining: 878ms
537:	learn: 0.2799395	total: 1.02s	remaining: 875ms
538:	learn: 0.2796695	total: 1.02s	remaining: 873ms
539:	learn: 0.2794378	total: 1.02s	remaining: 871ms
540:	learn: 0.2791440	total: 1.02s	remaining: 868ms
541:	learn: 0.2790670	total: 1.02s	remaining: 867ms
542:	learn: 0.2788688	total: 1.03s	remaining: 865ms
543:	learn: 0.2786795	to

684:	learn: 0.2574991	total: 1.31s	remaining: 603ms
685:	learn: 0.2572738	total: 1.31s	remaining: 601ms
686:	learn: 0.2571024	total: 1.31s	remaining: 599ms
687:	learn: 0.2568425	total: 1.32s	remaining: 598ms
688:	learn: 0.2565526	total: 1.32s	remaining: 596ms
689:	learn: 0.2565027	total: 1.32s	remaining: 594ms
690:	learn: 0.2561616	total: 1.32s	remaining: 592ms
691:	learn: 0.2560624	total: 1.32s	remaining: 590ms
692:	learn: 0.2558668	total: 1.33s	remaining: 588ms
693:	learn: 0.2556588	total: 1.33s	remaining: 586ms
694:	learn: 0.2553898	total: 1.33s	remaining: 584ms
695:	learn: 0.2553155	total: 1.33s	remaining: 582ms
696:	learn: 0.2552126	total: 1.33s	remaining: 581ms
697:	learn: 0.2550905	total: 1.34s	remaining: 579ms
698:	learn: 0.2550512	total: 1.34s	remaining: 577ms
699:	learn: 0.2548508	total: 1.34s	remaining: 575ms
700:	learn: 0.2547732	total: 1.34s	remaining: 573ms
701:	learn: 0.2545855	total: 1.35s	remaining: 571ms
702:	learn: 0.2543615	total: 1.35s	remaining: 570ms
703:	learn: 

917:	learn: 0.2251109	total: 1.81s	remaining: 161ms
918:	learn: 0.2249917	total: 1.81s	remaining: 160ms
919:	learn: 0.2248862	total: 1.81s	remaining: 158ms
920:	learn: 0.2247381	total: 1.81s	remaining: 156ms
921:	learn: 0.2245910	total: 1.82s	remaining: 154ms
922:	learn: 0.2245166	total: 1.82s	remaining: 152ms
923:	learn: 0.2244344	total: 1.82s	remaining: 150ms
924:	learn: 0.2244312	total: 1.82s	remaining: 148ms
925:	learn: 0.2242923	total: 1.82s	remaining: 146ms
926:	learn: 0.2241592	total: 1.83s	remaining: 144ms
927:	learn: 0.2241439	total: 1.83s	remaining: 142ms
928:	learn: 0.2240229	total: 1.83s	remaining: 140ms
929:	learn: 0.2238066	total: 1.84s	remaining: 138ms
930:	learn: 0.2237087	total: 1.84s	remaining: 136ms
931:	learn: 0.2235491	total: 1.84s	remaining: 134ms
932:	learn: 0.2233577	total: 1.84s	remaining: 132ms
933:	learn: 0.2231944	total: 1.85s	remaining: 130ms
934:	learn: 0.2230066	total: 1.85s	remaining: 128ms
935:	learn: 0.2228785	total: 1.85s	remaining: 126ms
936:	learn: 

,Model,Accuracy,Bal Acc.,Time
1,XGBoost,86.885246,86.525974,0.146937
2,CatBoost,86.885246,86.796537,3.642676
3,lr,85.245902,84.740260,0.059913
4,Skl GBM,83.606557,83.495671,0.199102
5,SVM,83.606557,82.954545,0.016993
6,Extra Trees,81.967213,81.439394,0.262125
7,Random Forest,81.967213,81.439394,0.352252
8,AdaBoost,81.967213,81.980519,0.195679
9,LightGBM,81.967213,81.439394,0.060469
10,Decision Tree,80.327869,80.194805,0.015232


In [ ]:
# Data enhancement based on gender with best features

In [68]:
def enhancement(data):
    gen_data =data
    for sex in data['sex'].unique():
        gender_data = gen_data[gen_data['sex']==sex]
        
        thalachh_std = gender_data['thalachh'].std()/10
        oldpeak_std = gender_data['oldpeak'].std()/10
        caa_std = gender_data['caa'].std()/10
        cp_std =gender_data['cp'].std()/10
        
        for i in gen_data[gen_data['sex']==sex].index:
            if np.random.randint(2) == 1:
                gen_data['thalachh'].values[i] += thalachh_std
            else:
                gen_data['thalachh'].values[i] -= thalachh_std
            
            if np.random.randint(2) == 1:
                gen_data['oldpeak'].values[i] += oldpeak_std
            else:
                gen_data['oldpeak'].values[i] -= oldpeak_std
            
            if np.random.randint(2) == 1:
                gen_data['caa'].values[i] += caa_std
            else:
                gen_data['caa'].values[i] -= caa_std
            
            if np.random.randint(2) == 1:
                gen_data['cp'].values[i] += cp_std
            else:
                gen_data['cp'].values[i] -= cp_std
                
    return gen_data

df.shape

        
        

(303, 14)

In [73]:
gen = enhancement(df)
gen.shape

(303, 14)

In [83]:
#random subset of the sample 
gen =gen.sample(gen.shape[0]//4)
gen = pd.concat((df,gen),axis=0)
gen.shape

(409, 14)

In [84]:
X,y = gen.drop(['output'],axis=1),gen.output

In [85]:

import time
X_train,X_test,y_train,y_test =model_selection.train_test_split(X,y,test_size=0.2,stratify = y,random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(X_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(X_test)# CODE HERE

    total_time = time.time() - start_time
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


### BEGIN SOLUTION

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[15:32:39] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.006392
0:	learn: 0.6881328	total: 1.5ms	remaining: 1.5s
1:	learn: 0.6852784	total: 2.77ms	remaining: 1.38s
2:	learn: 0.6816885	total: 4.01ms	remaining: 1.33s
3:	learn: 0.6773951	total: 5.3ms	remaining: 1.32s
4:	learn: 0.6737040	total: 6.53ms	remaining: 1.3s
5:	learn: 0.6700494	total: 7.76ms	remaining: 1.28s
6:	learn: 0.6673400	total: 8.8ms	remaining: 1.25s
7:	learn: 0.6640130	total: 9.96ms	remaining: 1.24s
8:	learn: 0.6610543	total: 11.1ms	remaining: 1.23s
9:	learn: 0.6589487	total: 12ms	remaining: 1.18s
10:	learn: 0.6559044	total: 13.5ms	remaining: 1.21s
11:	learn: 0.6533467	total: 14.8ms	remaining: 1.22s
12:	learn: 0.6490888	total: 16.1ms	remaining: 1.23s
13:	learn: 0.6447753	total: 17.4ms	remaining: 1.23s
14:	lea

180:	learn: 0.3707373	total: 240ms	remaining: 1.09s
181:	learn: 0.3695812	total: 242ms	remaining: 1.09s
182:	learn: 0.3687203	total: 243ms	remaining: 1.08s
183:	learn: 0.3683551	total: 244ms	remaining: 1.08s
184:	learn: 0.3670167	total: 245ms	remaining: 1.08s
185:	learn: 0.3661062	total: 246ms	remaining: 1.08s
186:	learn: 0.3650817	total: 248ms	remaining: 1.08s
187:	learn: 0.3640905	total: 249ms	remaining: 1.07s
188:	learn: 0.3628630	total: 250ms	remaining: 1.07s
189:	learn: 0.3618994	total: 252ms	remaining: 1.07s
190:	learn: 0.3612799	total: 253ms	remaining: 1.07s
191:	learn: 0.3604366	total: 254ms	remaining: 1.07s
192:	learn: 0.3594870	total: 255ms	remaining: 1.07s
193:	learn: 0.3586832	total: 257ms	remaining: 1.07s
194:	learn: 0.3576182	total: 258ms	remaining: 1.06s
195:	learn: 0.3562264	total: 260ms	remaining: 1.06s
196:	learn: 0.3558829	total: 261ms	remaining: 1.06s
197:	learn: 0.3550232	total: 262ms	remaining: 1.06s
198:	learn: 0.3543291	total: 264ms	remaining: 1.06s
199:	learn: 

436:	learn: 0.2360138	total: 585ms	remaining: 754ms
437:	learn: 0.2358204	total: 587ms	remaining: 753ms
438:	learn: 0.2354362	total: 588ms	remaining: 752ms
439:	learn: 0.2352330	total: 590ms	remaining: 751ms
440:	learn: 0.2348273	total: 591ms	remaining: 749ms
441:	learn: 0.2345180	total: 593ms	remaining: 748ms
442:	learn: 0.2339661	total: 594ms	remaining: 747ms
443:	learn: 0.2336844	total: 596ms	remaining: 746ms
444:	learn: 0.2335958	total: 597ms	remaining: 745ms
445:	learn: 0.2332383	total: 598ms	remaining: 743ms
446:	learn: 0.2329376	total: 600ms	remaining: 742ms
447:	learn: 0.2325746	total: 601ms	remaining: 741ms
448:	learn: 0.2322188	total: 603ms	remaining: 739ms
449:	learn: 0.2319944	total: 604ms	remaining: 738ms
450:	learn: 0.2317537	total: 605ms	remaining: 737ms
451:	learn: 0.2313721	total: 607ms	remaining: 736ms
452:	learn: 0.2312869	total: 608ms	remaining: 735ms
453:	learn: 0.2309774	total: 610ms	remaining: 734ms
454:	learn: 0.2308369	total: 612ms	remaining: 733ms
455:	learn: 

685:	learn: 0.1777834	total: 933ms	remaining: 427ms
686:	learn: 0.1773740	total: 935ms	remaining: 426ms
687:	learn: 0.1769879	total: 936ms	remaining: 425ms
688:	learn: 0.1768509	total: 938ms	remaining: 423ms
689:	learn: 0.1765872	total: 939ms	remaining: 422ms
690:	learn: 0.1764560	total: 941ms	remaining: 421ms
691:	learn: 0.1761469	total: 942ms	remaining: 419ms
692:	learn: 0.1759329	total: 944ms	remaining: 418ms
693:	learn: 0.1757615	total: 945ms	remaining: 417ms
694:	learn: 0.1756479	total: 947ms	remaining: 416ms
695:	learn: 0.1755653	total: 948ms	remaining: 414ms
696:	learn: 0.1752614	total: 950ms	remaining: 413ms
697:	learn: 0.1751413	total: 952ms	remaining: 412ms
698:	learn: 0.1751222	total: 954ms	remaining: 411ms
699:	learn: 0.1747781	total: 955ms	remaining: 409ms
700:	learn: 0.1744875	total: 957ms	remaining: 408ms
701:	learn: 0.1743051	total: 958ms	remaining: 407ms
702:	learn: 0.1742040	total: 960ms	remaining: 406ms
703:	learn: 0.1738837	total: 961ms	remaining: 404ms
704:	learn: 

941:	learn: 0.1371921	total: 1.28s	remaining: 78.8ms
942:	learn: 0.1370749	total: 1.28s	remaining: 77.4ms
943:	learn: 0.1368602	total: 1.28s	remaining: 76.1ms
944:	learn: 0.1367791	total: 1.28s	remaining: 74.7ms
945:	learn: 0.1367705	total: 1.28s	remaining: 73.3ms
946:	learn: 0.1367021	total: 1.29s	remaining: 72ms
947:	learn: 0.1366677	total: 1.29s	remaining: 70.6ms
948:	learn: 0.1365400	total: 1.29s	remaining: 69.3ms
949:	learn: 0.1364927	total: 1.29s	remaining: 67.9ms
950:	learn: 0.1363006	total: 1.29s	remaining: 66.6ms
951:	learn: 0.1361678	total: 1.29s	remaining: 65.2ms
952:	learn: 0.1359470	total: 1.29s	remaining: 63.9ms
953:	learn: 0.1357689	total: 1.3s	remaining: 62.5ms
954:	learn: 0.1356571	total: 1.3s	remaining: 61.2ms
955:	learn: 0.1353636	total: 1.3s	remaining: 59.8ms
956:	learn: 0.1351704	total: 1.3s	remaining: 58.5ms
957:	learn: 0.1351480	total: 1.3s	remaining: 57.1ms
958:	learn: 0.1350458	total: 1.3s	remaining: 55.8ms
959:	learn: 0.1348477	total: 1.3s	remaining: 54.4ms
96

,Model,Accuracy,Bal Acc.,Time
1,Random Forest,93.902439,93.723724,0.156021
2,Extra Trees,92.682927,92.612613,0.134405
3,Skl GBM,92.682927,92.612613,0.094080
4,Skl HistGBM,91.463415,91.261261,0.187280
5,XGBoost,91.463415,91.261261,0.073501
6,CatBoost,91.463415,91.261261,1.566291
7,LightGBM,90.243902,89.909910,0.027985
8,Decision Tree,89.024390,88.798799,0.014002
9,AdaBoost,89.024390,88.318318,0.088371
10,KNN,89.024390,89.279279,0.021766


In [138]:
gen['new_col']=gen[best_features].mean(axis=1)
gen['new_col']

0      0.380755
1      0.835980
2      0.329228
3      0.259384
4      0.134995
         ...   
202   -0.370678
185   -0.239789
182   -0.015311
291    0.461274
64     0.000284
Name: new_col, Length: 409, dtype: float64

In [139]:
gen.tail()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output,new_col
202,0.400752,1,0,1.049520,0.458709,0,0,-1.797762,1,0.315051,2,0,3,0,-0.370678
185,-1.143291,1,0,-1.120770,0.845219,0,0,0.252531,0,-1.211686,2,0,2,0,-0.239789
182,0.731619,0,0,-0.092738,1.618238,0,0,0.930665,0,-0.991910,2,0,2,0,-0.015311
291,0.400752,1,0,-1.006544,1.386332,0,2,-0.952337,0,2.797432,0,0,1,0,0.461274
64,0.400752,1,0,0.478391,-0.681494,1,0,1.203701,0,-1.202564,2,0,2,1,0.000284


In [140]:
X,y = gen.drop(['output'],axis=1),gen.output
import time
X_train,X_test,y_train,y_test =model_selection.train_test_split(X,y,test_size=0.2,stratify = y,random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(X_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(X_test)# CODE HERE

    total_time = time.time() - start_time
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


### BEGIN SOLUTION

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[16:26:25] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.006392
0:	learn: 0.6881328	total: 1.46ms	remaining: 1.46s
1:	learn: 0.6852784	total: 2.73ms	remaining: 1.36s
2:	learn: 0.6816885	total: 4.1ms	remaining: 1.36s
3:	learn: 0.6773951	total: 5.57ms	remaining: 1.39s
4:	learn: 0.6737040	total: 6.8ms	remaining: 1.35s
5:	learn: 0.6700494	total: 8.13ms	remaining: 1.35s
6:	learn: 0.6673400	total: 9.08ms	remaining: 1.29s
7:	learn: 0.6640130	total: 10.4ms	remaining: 1.28s
8:	learn: 0.6610543	total: 11.7ms	remaining: 1.29s
9:	learn: 0.6589487	total: 12.5ms	remaining: 1.24s
10:	learn: 0.6559044	total: 13.8ms	remaining: 1.24s
11:	learn: 0.6533467	total: 15.2ms	remaining: 1.25s
12:	learn: 0.6490888	total: 16.4ms	remaining: 1.25s
13:	learn: 0.6447753	total: 17.8ms	remaining: 1.26s
14

174:	learn: 0.3754656	total: 233ms	remaining: 1.1s
175:	learn: 0.3746038	total: 235ms	remaining: 1.1s
176:	learn: 0.3741836	total: 236ms	remaining: 1.1s
177:	learn: 0.3733340	total: 238ms	remaining: 1.1s
178:	learn: 0.3724281	total: 239ms	remaining: 1.09s
179:	learn: 0.3712849	total: 240ms	remaining: 1.09s
180:	learn: 0.3707373	total: 242ms	remaining: 1.09s
181:	learn: 0.3695812	total: 243ms	remaining: 1.09s
182:	learn: 0.3687203	total: 244ms	remaining: 1.09s
183:	learn: 0.3683551	total: 246ms	remaining: 1.09s
184:	learn: 0.3670167	total: 248ms	remaining: 1.09s
185:	learn: 0.3661062	total: 249ms	remaining: 1.09s
186:	learn: 0.3650817	total: 250ms	remaining: 1.09s
187:	learn: 0.3640905	total: 252ms	remaining: 1.09s
188:	learn: 0.3628630	total: 253ms	remaining: 1.09s
189:	learn: 0.3618994	total: 255ms	remaining: 1.08s
190:	learn: 0.3612799	total: 256ms	remaining: 1.08s
191:	learn: 0.3604366	total: 258ms	remaining: 1.08s
192:	learn: 0.3594870	total: 259ms	remaining: 1.08s
193:	learn: 0.35

422:	learn: 0.2411662	total: 575ms	remaining: 785ms
423:	learn: 0.2408996	total: 577ms	remaining: 784ms
424:	learn: 0.2405589	total: 579ms	remaining: 783ms
425:	learn: 0.2402914	total: 580ms	remaining: 782ms
426:	learn: 0.2398731	total: 582ms	remaining: 781ms
427:	learn: 0.2396597	total: 583ms	remaining: 779ms
428:	learn: 0.2391360	total: 584ms	remaining: 778ms
429:	learn: 0.2386870	total: 586ms	remaining: 777ms
430:	learn: 0.2383372	total: 587ms	remaining: 776ms
431:	learn: 0.2377642	total: 589ms	remaining: 774ms
432:	learn: 0.2373144	total: 590ms	remaining: 773ms
433:	learn: 0.2371231	total: 592ms	remaining: 772ms
434:	learn: 0.2366060	total: 593ms	remaining: 771ms
435:	learn: 0.2363433	total: 595ms	remaining: 769ms
436:	learn: 0.2360138	total: 596ms	remaining: 768ms
437:	learn: 0.2358204	total: 598ms	remaining: 768ms
438:	learn: 0.2354362	total: 601ms	remaining: 768ms
439:	learn: 0.2352330	total: 602ms	remaining: 766ms
440:	learn: 0.2348273	total: 604ms	remaining: 765ms
441:	learn: 

662:	learn: 0.1818191	total: 927ms	remaining: 471ms
663:	learn: 0.1816491	total: 929ms	remaining: 470ms
664:	learn: 0.1815359	total: 931ms	remaining: 469ms
665:	learn: 0.1813429	total: 932ms	remaining: 467ms
666:	learn: 0.1812318	total: 934ms	remaining: 466ms
667:	learn: 0.1808060	total: 935ms	remaining: 465ms
668:	learn: 0.1807078	total: 937ms	remaining: 463ms
669:	learn: 0.1804839	total: 938ms	remaining: 462ms
670:	learn: 0.1802402	total: 941ms	remaining: 461ms
671:	learn: 0.1800782	total: 942ms	remaining: 460ms
672:	learn: 0.1797426	total: 944ms	remaining: 459ms
673:	learn: 0.1796634	total: 945ms	remaining: 457ms
674:	learn: 0.1795445	total: 947ms	remaining: 456ms
675:	learn: 0.1793007	total: 949ms	remaining: 455ms
676:	learn: 0.1791383	total: 951ms	remaining: 454ms
677:	learn: 0.1789499	total: 953ms	remaining: 452ms
678:	learn: 0.1788018	total: 954ms	remaining: 451ms
679:	learn: 0.1786786	total: 956ms	remaining: 450ms
680:	learn: 0.1784845	total: 957ms	remaining: 448ms
681:	learn: 

899:	learn: 0.1434462	total: 1.27s	remaining: 142ms
900:	learn: 0.1433698	total: 1.28s	remaining: 140ms
901:	learn: 0.1433037	total: 1.28s	remaining: 139ms
902:	learn: 0.1432207	total: 1.28s	remaining: 137ms
903:	learn: 0.1430435	total: 1.28s	remaining: 136ms
904:	learn: 0.1427837	total: 1.28s	remaining: 135ms
905:	learn: 0.1425446	total: 1.28s	remaining: 133ms
906:	learn: 0.1423464	total: 1.28s	remaining: 132ms
907:	learn: 0.1421865	total: 1.29s	remaining: 130ms
908:	learn: 0.1420866	total: 1.29s	remaining: 129ms
909:	learn: 0.1418964	total: 1.29s	remaining: 128ms
910:	learn: 0.1417051	total: 1.29s	remaining: 126ms
911:	learn: 0.1416230	total: 1.29s	remaining: 125ms
912:	learn: 0.1414515	total: 1.29s	remaining: 123ms
913:	learn: 0.1412698	total: 1.3s	remaining: 122ms
914:	learn: 0.1409961	total: 1.3s	remaining: 121ms
915:	learn: 0.1409209	total: 1.3s	remaining: 119ms
916:	learn: 0.1407552	total: 1.3s	remaining: 118ms
917:	learn: 0.1404913	total: 1.3s	remaining: 116ms
918:	learn: 0.140

,Model,Accuracy,Bal Acc.,Time
1,Extra Trees,93.902439,93.723724,0.132853
2,Random Forest,92.682927,92.372372,0.157134
3,Skl GBM,92.682927,92.612613,0.088043
4,Skl HistGBM,91.463415,91.261261,0.168612
5,XGBoost,91.463415,91.261261,0.075063
6,CatBoost,91.463415,91.261261,1.629579
7,LightGBM,90.243902,89.909910,0.026411
8,AdaBoost,89.024390,88.318318,0.082257
9,KNN,89.024390,89.279279,0.008011
10,Decision Tree,87.804878,87.447447,0.012669


In [142]:
results_ord['Accuracy'].iloc[0]

93.90243902439023